<a href="https://colab.research.google.com/github/KingLoczek/rekrutacja-zima-2023/blob/main/Zadanie2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Wczytanie bibliotek
!pip install e2cnn
import os
from google.colab import drive
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from e2cnn import gspaces
from e2cnn import nn as gnn
from e2cnn import group

In [2]:
#Pobranie zawartosci folderu z dysku google (Retina Blood Vessel)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Ścieżki do folderów
test_images_folder = '/content/drive/My Drive/Colab Notebooks/Data/test/image'
test_mask_folder = '/content/drive/My Drive/Colab Notebooks/Data/test/mask'
train_images_folder = '/content/drive/My Drive/Colab Notebooks/Data/train/image'
train_mask_folder = '/content/drive/My Drive/Colab Notebooks/Data/train/mask'


# Sprawdź, czy folder istnieje
if os.path.exists(test_mask_folder):
    # # Wyświetl obrazy PNG
    # png_files = [f for f in os.listdir(test_mask) if f.endswith('.png')]
    # if png_files:
    #     for png_file in png_files:
    #         image_path = os.path.join(test_mask, png_file)
    #         img = Image.open(image_path)
    #         display(img)
    # else:
    #     print("Brak plików PNG w folderze.")
    print("Wszystko git.")
else:
    print("Podana ścieżka nie istnieje.")

Wszystko git.


In [4]:
# Utwórz własny dataset na podstawie folderów z obrazami i etykietami
class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform

        # Wczytaj nazwy plików obrazów i etykiet
        self.images = os.listdir(image_folder)
        self.masks = os.listdir(mask_folder)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.images[idx])
        mask_name = os.path.join(self.mask_folder, self.masks[idx])

        # Wczytaj obraz i etykietę
        img = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('L')  # Konwertuj do skali szarości

        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)

        return img, mask

# Przygotuj transformacje danych
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Utwórz instancje datasetów dla zbiorów treningowego, walidacyjnego i testowego
train_dataset = CustomDataset(train_images_folder, train_mask_folder, transform=transform)
test_dataset = CustomDataset(test_images_folder, test_mask_folder, transform=transform)

# Podziel zbiór treningowy na treningowy i walidacyjny
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42)

# Utwórz DataLoader dla zbiorów treningowego, walidacyjnego i testowego
train_loader = DataLoader(dataset=train_dataset, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(train_indices))
val_loader = DataLoader(dataset=train_dataset, batch_size=32, sampler=torch.utils.data.SubsetRandomSampler(val_indices))
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# # Wyświetl przykładowe obrazy z datasetu
# sample = next(iter(train_loader))
# images, masks = sample

# # Wyświetl kilka przykładowych obrazów i etykiet
# for i in range(5):
#     display(transforms.ToPILImage()(images[i]))
#     display(transforms.ToPILImage()(masks[i]))